# Toronto neighborhood analysis project

##Importing libraries

In [181]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Step 1. Downloading the neighborhood data
In this step we will download Toronto neighborhood data and create a dataframe for further manipulations

###Scraping data from WikiPedia
We will use the table from following link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Read the table and save it as a dataframe

In [182]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


###Create a final dataframe

Let's fined the rows that with "Not assigned" Boroughs...

In [0]:
ix=df[df.Borough == 'Not assigned'].index

...and drop them

In [184]:
# Drop rows
df.drop(ix, inplace=True)
# Sort by Postal code
df.sort_values(by=['Postal Code'], inplace=True)
# Reset index
df.reset_index(inplace=True, drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [193]:
df.columns

Index(['Postal Code', 'Borough', 'Neighborhood'], dtype='object')

### Saving the dataframe 
DEAR EVALUATORS, IGNORE THE CODE BELOW.

I'm using Google Colab to run my Notebook and next lines of code are used to save the results in Google Drive

In [0]:
df.to_csv('df-toronto-neihgborhoods.csv', index=False)
df.to_excel('df-toronto-neihgborhoods.xlsx', index=False)

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!cp -r df-toronto-neihgborhoods.xlsx drive/My\ Drive/Colab/ML-clustering-toronto-neighborhoods/df-toronto-neihgborhoods.xlsx
#!cp -r df-toronto-neihgborhoods.csv drive/My\ Drive/Colab/ML-clustering-toronto-neighborhoods/df-toronto-neihgborhoods.csv

#Step 2: Get latitude and longitude coordinates
Next we get lat and long coordinates for each neighborhood.

### First we gonna read the csv file with Ontario postal codes and related coordinates

In [189]:
postal_df =  pd.read_csv('http://cocl.us/Geospatial_data')
postal_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Next we gonna merge newly obtained data with our dataframe based on Postal code matching

In [200]:
df = df.merge(postal_df, how='left', on='Postal Code', copy=True)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Saving data to Google drive

In [0]:
df.to_csv('df-toronto-neihgborhoods-postalcodes.csv', index=False)
df.to_excel('df-toronto-neihgborhoods-postalcodes.xlsx', index=False)

In [202]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#!cp -r df-toronto-neihgborhoods-postalcodes.xlsx drive/My\ Drive/Colab/ML-clustering-toronto-neighborhoods/df-toronto-neihgborhoods-postalcodes.xlsx
#!cp -r df-toronto-neihgborhoods-postalcodes.csv drive/My\ Drive/Colab/ML-clustering-toronto-neighborhoods/df-toronto-neihgborhoods-postalcodes.csv